# Generación del escenario post desastre

In [1]:
# Bibliotecas de uso general en el cuaderno
from myutils import *
import cugraph
import pickle
import os

## Datos generales (PLTBs y hospitales)

In [2]:
# Directorio de datos generales
path = "../GeoData/"
escenario = '0'
out = "../GeoData/Instances/Instance_" + escenario + ".pkl"

In [3]:
# Carga el grafo de la red de transporte de la Ciudad de México
g = ox.load_graphml(filepath = path + "graph_transport.graphml")
gdf_nodes, gdf_edges = ox.graph_to_gdfs(g, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)

In [4]:
# Carga las pltbs
pltbs = gpd.read_file(path + 'PLTBs.gpkg', layer = 'PLTBs_nodes')

# Serializa los datos de tipo lista
pltbs['streets'] = pltbs['streets'].apply(json.loads)
pltbs['oneway'] = pltbs['oneway'].apply(json.loads)
pltbs['length'] = pltbs['length'].apply(json.loads)
pltbs['capacity'] = pltbs['capacity'].apply(json.loads)
pltbs['grouped'] = pltbs['grouped'].apply(json.loads)
pltbs_grupos= [item for sublist in [ i for i in pltbs['grouped']] for item in sublist]
pltbs_nodos = list(pltbs['node'])

print('Grafo de transporte y PLTBs cargados')

Grafo de transporte y PLTBs cargados


In [5]:
# Identifica todos los nodos que no se puedne alcanzar desde una PLTB
voronoi_pltbs = nx.voronoi_cells(g, pltbs_nodos, weight='weight')
unreachable = voronoi_pltbs["unreachable"]
del(voronoi_pltbs)

# Elimina los nodos que no se pueden alcanzar desde una PLTB
g_u = g.copy()
g_u.remove_nodes_from(unreachable)

In [6]:
# Carga el archivo con la ubicación de los hospitales
hospitales = gpd.read_file(path + 'Points.gpkg', layer = 'hospitales')

# Identifica hospitales de interés
hospitales_viables = hospitales.query("camas_en_a > 10")

# Obtiene las coordenadas
hospitales_coords = hospitales_viables.get_coordinates()

# Obtiene las aristas más cercanas en el grafo de la red de transporte "g"
aristas_hospitales = list(set(ox.nearest_edges(g_u, hospitales_coords.x, hospitales_coords.y)))

# añade las aristas en doble sentido, son las que se van a descartar como PLTBs
aristas_hospitales_d = copy(aristas_hospitales)
aristas_hospitales_d += [(v,u,k) for u,v,k in aristas_hospitales_d if (v,u,k) in g_u.edges.keys()]

# Añade las aristas vecinas también
# aristas_hospitales_vec = [(u,v,0) for a in aristas_hospitales_d for u,v in g.edges(a[0]) ]

## Datos propios del incidente

En esta sección se obtienen datos que varían dependiendo del incidente, como la posición de los puntos de demanda o los datos de tráfico

In [7]:
# Carga el archivo con la ubicación de los puntos de atención
colapsos = gpd.read_file(path + 'Points_instances.gpkg', layer = 'Instance_' + escenario)

# Obtiene las coordenadas
colapsos_coords = colapsos.get_coordinates()

# Obtiene las aristas más cercanas en el grafo de la red de transporte "g", 
# en este caso es importante mantener el orden, por eso se usa una lista en lugar de un set
aristas_colapsos = ox.nearest_edges(g_u, colapsos_coords.x, colapsos_coords.y)

# Añade las aristas en doble sentido, son las que se van a bloquear
aristas_colapsos_b = list(set(copy(aristas_colapsos)))
aristas_colapsos_b += [(v,u,k) for u,v,k in aristas_colapsos_b if (v,u,k) in g_u.edges.keys()]

# Las aristas bloqueadsa y sus vecinas también serán descartadas como PLTBs
aristas_colapsos_d = copy(aristas_colapsos_b)
aristas_colapsos_d += [(u,v,0) for a in aristas_colapsos_d for u,v in g.edges(a[0])]

# Elimina el grafo g_u
del(g_u)

### Lectura de datos de tráfico previamente obtenidos

Se lee el archivo .tar con la información de tráfico para cada arista del grafo

In [8]:
traffic_files = [os.path.join(path + 'Traffic/', file) for file in os.listdir(path + 'Traffic/') if file.endswith("_traffic_data.tar")]
traffic_files.sort()
print(traffic_files[int(escenario)])
traffic_data = pd.read_pickle(traffic_files[int(escenario)])

GeoData/Traffic/traffic_flow_16_2023-10-30_13-01-01_traffic_data.tar


### Alternativamente: obtención de los datos de tráfico al momento en el que se ejecuta la celda

En esta celda se carga el polígono y se descargan las teselas necesarias para capturar el tráfico. De manera predeterminada únicamente se almacenan los datos de tráfico como un archivo .tar, pero se pueden cambiar los parámetros para almacenar los pasos interemedios, como geotiff o las teselas independientes como diccionario de python. En caso de querer guardar, también se puede almacenar en RGBA y hacer la conversión a monobanda en el último paso.

In [9]:
# # Carga el polígono de la ciudad de méxico, aplica un buffer de 100 mts y lo proyecta a WGS84
# cdmx = gpd.read_file(path + '/Polygons.gpkg', layer='CDMX', encoding='utf-8')
# polygon = cdmx.geometry[0].buffer(100)
# cdmx['geometry'] = polygon
# cdmx.to_crs(4326, inplace=True)
# polygon = cdmx.geometry[0]

# # Api key de TomTom
# apiKey = '5mcWU77vx9s9EaLyqfw61oyfeoe3MAl8'
# zoom = 16
# imsize = 512

# # Obtiene las teselas
# tiles, tiles_data  = await get_traffic_tiles(polygon, zoom, imsize, apiKey, save = False, 
#                                       saveGeometry = False, path=path + 'Traffic/', rgba=False)

# name = tiles_data['name']

# # # Las guarda en un archivo de tipo raster
# # save_geotif(tiles, tiles_data, path=path + 'Traffic/', name=name)

# # Obtiene el tráfico para cada arista según las teselas
# traffic_data = get_traffic_data(g, tiles, tiles_data, save = True, path=path + 'Traffic/' , name=name)
# del(tiles)


### Añade la información de tráfico al grafo

In [10]:
# Crear una columna con el tráfico en cada arista
gdf_edges = gdf_edges.assign(traffic= traffic_data)

## Bloqueo de aristas 

En esta sección se bloquean las aristas del grafo en donde hay un punto de demanda

In [12]:
# Crea una columna para indicar la penalización por bloqueo

weight_max = 1e9

block = lambda x: 1 if (x[0],x[1],0) in aristas_colapsos_b else 0
gdf_edges['blocked'] = gdf_edges.index.to_series().apply(block)

bby_colapsos = gdf_edges["blocked"].sum()
print(f'Se bloquearon {bby_colapsos} aristas por colapsos')


gdf_edges['blocked'] += ((gdf_edges['traffic'] == 5) | (gdf_edges['traffic'] == 6))*1


bby_traffic = gdf_edges["blocked"].sum() - bby_colapsos
print(f'Se bloquearon {bby_traffic} aristas por tráfico')



Se bloquearon 29 aristas por colapsos
Se bloquearon 155 aristas por tráfico


In [13]:
# Pondera las aristas según el tráfico, momentaneamente se asigna un valor de 0 a las aristas bloqueadas
map_vel = {1:1, 2: 2, 3: 4, 4: 6.7, 5: 0, 6: 0}
gdf_edges['weight'] = gdf_edges.apply(lambda x: x['length'] * map_vel[ x['traffic']], axis=1)

# Asigna el paso de todas las aristas bloqueadas a 1e9
gdf_edges['weight'] = gdf_edges['weight'] * (1-gdf_edges['blocked']) + weight_max * gdf_edges['blocked']

## Descarte de PLTBs

In [14]:
# Aristas a descartar como PLTBs por no tener tráfico libre
aristas_trafico_d = [(u,v,k) for u,v,k in gdf_edges[gdf_edges['traffic'] > 2].index.to_series()]

In [15]:
# Aristas que se deben descartar como PLTBs
aristas_a_descartar = list(set(aristas_hospitales_d + aristas_colapsos_d))
pltbs_a_descartar = set()

for u,v,k in aristas_a_descartar:
    
    # Si alguno de los nodos está en la lista de PLTBs agrupadas, los nodos head del grupo(s) se agregan a la lista de PLTBs bloqueadas 
    if u in pltbs_grupos or v in pltbs_grupos:
        block = list(pltbs[pltbs.apply( lambda x: u in x['grouped'] or v in x['grouped'], axis = 1)].get('node'))
        pltbs_a_descartar = pltbs_a_descartar.union(set(block))
 
    # Descarta pltbs que no tienen sucesores (normalmene en los extremos del bbox)
    if len(list(g.successors(u))) == 0:
        pltbs_a_descartar.add(u)
    if len(list(g.successors(v))) == 0:
        pltbs_a_descartar.add(v)
        
# En el caso del tráfico, se descartan las aristas que no tienen tráfico libre 
for u,v,k in aristas_trafico_d:
    
    # Si el nodo v es una PLTB, se agrega a la lista de PLTBs a descartar
    if v in pltbs_nodos:
        pltbs_a_descartar.add(v)
    if u in pltbs_nodos:
        pltbs_a_descartar.add(u)

In [16]:
print('pltbs que quedan descartadas:')
print('(', end='')
for b in pltbs_a_descartar:
    print(b,end=', ')
print(')')
print('Total de PLTBs descartadas:', len(pltbs_a_descartar))

pltbs que quedan descartadas:
(268902400, 30341126, 380231687, 3309034509, 30341135, 30594063, 7960991768, 933696538, 30341146, 933696542, 271534110, 8558343205, 8338268211, 4073763912, 562619468, 2746797139, 271464534, 5839130713, 5649404007, 436858984, 1108181104, 842912886, 303558774, 5653883003, 4551550084, 30385293, 681921679, 4089076880, 337140883, 506862742, 30594200, 387995802, 873739419, 506045597, 3003083934, 305066155, 274206896, 385609908, 5849129142, 3292767422, 299506880, 7771318470, 268803285, 61470942, 270978274, 271571175, 419331307, 2692221169, 1795200246, 272396535, 6473513209, 386274555, 10822491389, 272197887, 345208065, 268451077, 274201868, 270484752, 298935574, 299506978, 8743150883, 271540516, 386277669, 8088885547, 1420541228, 270979376, 912837937, 30880050, 302753074, 274429234, 712767794, 683899187, 1100471606, 268803385, 268803388, 31319361, 268508494, 500122960, 269832531, 712767830, 3035173206, 276496729, 1119606105, 401433947, 372949348, 270105957, 14205

In [17]:
# Vuelve a forma el grafo de la red de transporte
g = ox.graph_from_gdfs(gdf_nodes, gdf_edges)

# Si se desea guardar el grafo bloqueado
# save_graph_geopackage(g, filepath='../GeoData/graph_transport_bloqueado.gpkg', layer='grafo_bloqueado', encoding='utf-8')

## Cálculo de matríz origen destino

Ya que los puntos de destino son significativamente menos que los de origen, se invierte el grafo para encontrar la ruta desde todos los vertices hasta cada uno de los puntos de atención. 

In [18]:
# Grafo de transporte invertido
gi = g.reverse()
g_i_nx = nx.DiGraph(gi)

edges_i = nx.to_pandas_edgelist(g_i_nx)
edges_i = edges_i[['source','target','weight']]

g_i_cuda = cugraph.Graph(directed=True)
g_i_cuda.from_pandas_edgelist(edges_i, source='source', destination='target', edge_attr='weight')

In [19]:
print('Total de aristas omitidas: ',len(gdf_edges)-len(g_i_cuda.edges()))

Total de aristas omitidas:  1580


### Define los origenes y destinos en términos de los ids de los nodos del grafo de transporte

In [20]:
# Identifica los orígenes y destinos vecinos, considerando las vías bloqueadas
origenes = [ p for p in pltbs_nodos if p not in pltbs_a_descartar]
destinos = colapsos['num']

# Identifica para cada punto de colapso "c" su arista con puntos vecinos "u,v"
# con los que se puede llegar a "c" desde "u" o "v"
colapsos_vecinos = dict([(c,[u,v]) for c, (u,v,k) in zip(destinos, aristas_colapsos )])

# Identifica todos los puntos de destino sin repetir
destinos_vecinos = set([i for u,v,k in aristas_colapsos for i in [u,v]])


In [21]:
# Crea un diccionario con los indices de los nodos de origen y destino
origen_i_to_node = dict([(i,o) for i,o in enumerate(origenes)])
origen_node_to_i = dict([(o,i) for i,o in enumerate(origenes)])

# Mapeos de indices "num" a nodos y viceversa
destino_num_to_node = colapsos_vecinos
destino_node_to_num = dict([(i,c) for c, (u,v,k) in zip(destinos, aristas_colapsos ) for i in [u,v]])

# Mapeos de indices "num" a índices reales y viceversa
destino_i_to_num = dict([(i, d) for i,d in enumerate(destinos)])
destino_num_to_i = dict([(d, i) for i,d in enumerate(destinos)])

In [22]:
pltbs.index = pltbs['node']
colapsos.index = colapsos['num']
# Toma la capacidad total/2 de cada PLTB
capacidades = [pltbs.loc[i]['t_capacity']/2 for i in origenes]


### Obtiene las rutas desde cada punto de origen hasta cada destino

Se almacena en un diccionario, en donde la llave son los destinos y el valor es un dataframe con el resultado del algoritmo de Dijkstra, de dondes e puede obtener la ruta y el costo de llegada desde todos los puntos del grafo de transporte

In [23]:
rutas = dict()

# Calcula las rutas más cortas para cada punto de atención usando el grafo invertido
# y dijkstra, las almacena en un diccionario cuya llave es el punto de destino

for i,d in enumerate(destinos_vecinos):
    df_rutas = cugraph.sssp(g_i_cuda, d, edge_attr='weight')
    df_rutas.set_index('vertex', inplace=True)
    rutas[d] = df_rutas.round(4).to_pandas()

print('costo calculado para cada punto de atención')

costo calculado para cada punto de atención


In [24]:
# Versión con networkx
# rutas_nx = dict()
# for i,d in enumerate(destinos_vecinos):
#     rutas_nx[d] = nx.single_source_dijkstra(g_i_nx, d, weight='weight')[0]


In [25]:
# En esta celda, se verifica que es equivalente invertir el grafo para calcular las rutas con los puntos de atención como origen

# g_nx = nx.DiGraph(g)

# Convierte los grafo de transporte a grafos de NetworkX
# edges = nx.to_pandas_edgelist(g_nx)
# edges = edges[['source','target','weight']]

# Crea los grafos de transporte en cuGraph
# g_cuda = cugraph.Graph(directed=True)
# g_cuda.from_pandas_edgelist(edges, source='source', destination='target', edge_attr='weight')

# nodos_ejemplo = 5
# rutas2 = dict()
# for i,o in enumerate(origenes[0:nodos_ejemplo]): # solo para 10 nodos
#     df_rutas2 = cugraph.sssp(g_cuda, o, edge_attr='weight')
#     df_rutas2.set_index('vertex', inplace=True)
#     rutas2[o] = df_rutas2

# for i in range(nodos_ejemplo):
#     o = origen_i_to_node[i]
#     for j in range(nodos_ejemplo):
#         j = destino_i_to_node[j]
#         if i != j:
#             print('origen:',o, 'destino:',d)
#             print('costo con grafo normal:', rutas2[o].loc[d]['distance'])
#             print('costo con grafo invertido:', rutas[d].loc[o]['distance'])
#             print('------------------------------------',end='\n\n')


### Obtención de la matriz Origen Destino

En esta matriz, las filas son las PLTBS y las columnas los colapsos

filas: PLTBS
columnas: Colapsos

In [26]:
# Crea la matriz OD
matriz_OD = np.zeros((len(origenes), len(destinos)))

# Para cada foco de atención "d":
for i, d in enumerate(destinos):
    
    # Identifica cuáles son sus vecinos "u,v" por los que se puede llegar a él
    vecino_u = colapsos_vecinos[d][0]
    vecino_v = colapsos_vecinos[d][1]
    
    # De las rutas calculadas, obtiene la distancia de cada origen a "u" y a "v"
    distancias_u = rutas[vecino_u].loc[origenes]['distance'].rename('distancia_u')
    distancias_v = rutas[vecino_v].loc[origenes]['distance'].rename('distancia_v')
    
    # Sustituye los valores mayores a 10e20 por 10e20, esto deja las rutas que llegan con un valor normal
    # distancias_u < weight_max
    # o las rutas que de plano son imposibles, 
    # distancias_u > 10e20
    # quita las rutas que llegan por aristas bloqueadas
    
    distancias_u = distancias_u*(distancias_u < weight_max) #+ 10e20*(distancias_u > weight_max)
    distancias_v = distancias_v*(distancias_v < weight_max) #+ 10e20*(distancias_v > weight_max)
    
    # Encuentra la distancia máxima para llegar a "u" o a "v desde cada origen
    distancias_d = pd.concat([distancias_u, distancias_v], axis=1).max(axis=1)
    
    # Agrerga la columna a la matriz OD
    matriz_OD[:,i] = distancias_d.to_numpy()

# Sustituye valores de 0 por 10e20
matriz_OD += 10e20*(matriz_OD == 0)

### Verificación de valores

FInalmente se puede consultar una distancia específica entre dos puntos para comparar con la solución de otro software como Qgis

In [27]:
# Se identifica la PLTB que ocupa el lugar 45 en la lista de PLTBs
o_test = 885813813

# se identifican los destinos del colapso num = 28
d_test = colapsos_vecinos[10]
print("origen: ", o_test, "destinos: ", d_test)

origen:  885813813 destinos:  [6426394741, 6426394740]


In [28]:
# rutas["llega_a:"].loc["desde:"]
print(rutas[d_test[0]].loc[o_test])
print(rutas[d_test[1]].loc[o_test])

distance       3.586574e+04
predecessor    2.343332e+09
Name: 885813813, dtype: float64
distance       1.000036e+09
predecessor    2.343332e+09
Name: 885813813, dtype: float64


### Obtención de ruta

A partir del diccionario de rutas se obiene la ruta completa

In [29]:
a = get_route_from_df(rutas[d_test[0]],o_test)

print('(',end='')
for i in a[1]:
    print(i,end=', ')

print(')')

(6426394741, 365071548, 3033876340, 3693891318, 3033876345, 365071552, 332717008, 7956149575, 7943251860, 6175369860, 2910122897, 7956275181, 7956275182, 6426394924, 4252847523, 7956324404, 332716988, 7952121582, 332716983, 6426395033, 7964446905, 332716978, 1494127324, 6175520921, 6169114439, 332716962, 332716960, 7955058384, 332718550, 6437158225, 7954949365, 7939506963, 7939506964, 7939801315, 274979571, 8999000205, 7942905495, 274980144, 275031858, 275029510, 7960666645, 274980763, 274997681, 274996697, 691468110, 8643513237, 1695838148, 1695838176, 7942812603, 274996692, 274994785, 274994782, 1015469877, 1015469855, 4087232145, 1695838281, 274993488, 274993763, 2716409563, 2716409562, 6166748670, 2716391677, 2716409560, 336596993, 2716391662, 343611357, 6298545954, 331992529, 331992523, 1397756675, 276072676, 276077874, 1428545162, 1428545171, 276069803, 276070529, 276070033, 276070334, 344410767, 4086720820, 270607846, 270607845, 274920033, 274929697, 274929236, 274928739, 274920

## Determina la demanda de cada colapso

Para determinar la demanda, toma el porcentaje de víctimas en cada uno de los colapsos, y toma ese porcentaje del total de ambulancias disponibles. Además garantiza que cada punto tenga al menos una demanda de 1

In [30]:
victimas = np.array([colapsos.loc[destino_i_to_num[i]]['victimas'] for i in range(len(destinos))])

# Numero total de ambulancias disponibles
num_ambulancias = 250 
demanda = (victimas/np.sum(victimas)) * (num_ambulancias - len(victimas))
demanda = np.round(demanda).astype(int) + 1
demanda = demanda.tolist()
print('Demanda proporcional: ',sum(demanda))


if sum(demanda) > num_ambulancias:
    print('Demanda proporcional ajustada: ',sum(demanda))
    
print(demanda)

Demanda proporcional:  251
Demanda proporcional ajustada:  251
[44, 6, 20, 5, 5, 25, 20, 6, 15, 5, 6, 15, 5, 5, 13, 20, 6, 15, 15]


## Almacenamiento de la instancia

Posteriormente se almacenan las variables que representan a esta instancia

In [31]:
instancia = {'origenes': origenes,
             'destinos': destinos,
             
             'origen_i_to_node': origen_i_to_node,
             'origen_node_to_i': origen_node_to_i,
             
             'destino_i_to_num': destino_i_to_num,
             'destino_num_to_i': destino_num_to_i,
             
             'destino_num_to_node': destino_num_to_node,
             'destino_node_to_num': destino_node_to_num,
             
             'capacidades': capacidades,
             'demanda': demanda,
             
             'matriz_OD': matriz_OD,
             'rutas': rutas
             }


In [32]:
# with open(out, 'wb') as f:  
#     pickle.dump(instancia, f)
#     f.close()

# with open(out, 'rb') as f:
#     instance = pickle.load(f) 
#     f.close()
